In [1]:
minmaxInterval = 2000

## open port

In [2]:
!echo 'quzinee1!'| sudo -S chmod a+rw /dev/ttyUSB1

[sudo] password for terry: chmod: cannot access '/dev/ttyUSB1': No such file or directory


## BASIC USAGE OF XL320 CLASS

In [3]:
import os, time
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from math import pi as PI
DEG90 = PI/2
PROPORTION_MATRIX = np.array(
    [
        [1,0.1,0.4,0.4, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.1,1,0.4,0.4, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,1,0.1, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,0.1,1, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        
        [0,0,0,0,  1,0.1,0.4,0.4, 0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.1,1,0.4,0.4,  0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.4,0.4,1,0.1, 0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.4,0.4,0.1,1, 0,0,0,0, 0,0,0,0],
        
        [0,0,0,0, 0,0,0,0, 1,0.1,0.4,0.4, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.1,1,0.4,0.4, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,1,0.1, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,0.1,1, 0,0,0,0],
        
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 1,0.1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.1,1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.4,0.4,1,0.1],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.4,0.4,0.1,1],
        ])
PROPORTION_MATRIX = PROPORTION_MATRIX[:4,:4]
PROPORTION_MATRIX.shape

(4, 4)

In [4]:
import sys
sys.path.append(".")
""" FOR DYNAMIXEL """
from model.class_xc330 import xc330
""" FOR MODERN DRIVER """
import roslib; roslib.load_manifest('ur_driver')
import rospy
import actionlib
from control_msgs.msg import *
from trajectory_msgs.msg import *
from sensor_msgs.msg import JointState

from class_vicon_marker import VICON
from class_vicon_base import ViconBase

class OneFinger(object):
    def __init__(self,):
        rospy.init_node("DEMO")
        self.JOINT_NAMES = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
                            'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
        self.client      = None
        
        self.xc          = xc330('SOFTROBOT', _USB_NUM=0)
        
    def init_xc(self,initpos):
        self.xc.connect()
        self.xc.set_operatingmode([4])

        self.xc.set_torque([0])
        time.sleep(2)
        self.xc.set_torque([1])
        print ("TORQUE ON")

        currpos = self.xc.get_currpos()
        minmaxInterval = 5000
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)

        if initpos is None:
            initpos = currpos; print("No initpos Found")
            self.set_init_pos()
            return currpos

        self.xc.set_goalposcluster(initpos,1)

        print("INITIALIZE POSITION")
        currpos = self.xc.get_currpos()
        print(currpos)

        minmaxInterval = minmaxInterval
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)
        self.set_init_pos()
        return currpos
        
    def set_init_pos(self):
        self.initpos = self.xc.get_currpos()
    # def move_xc(self,pos):
    #     pos = self.initpos + np.array(pos)* np.array([1,-1,-1,1])
    #     self.xc.set_goalposcluster(pos,1)
        
    def pmove_xc(self,pos):
        pos = self.initpos + PROPORTION_MATRIX @ np.array(pos) * np.array([1,-1,-1,1])
        self.xc.set_goalposcluster(pos,1)
        
    def move_arm_speed(self, joints, speed):
        try: 
            q = joints
            g = FollowJointTrajectoryGoal()
            g.trajectory = JointTrajectory()
            g.trajectory.joint_names = self.JOINT_NAMES
            joint_states = rospy.wait_for_message("joint_states", JointState)
            joints_pos   = joint_states.position

            diff = np.array(joints_pos) - np.array(q)

            assert speed <= 1e-1 
            movetime = np.linalg.norm(diff)/speed

            g.trajectory.points = [
                JointTrajectoryPoint(positions=joints_pos, velocities=[0]*6, time_from_start=rospy.Duration(0.0)),
                JointTrajectoryPoint(positions=q, velocities=[0]*6, time_from_start=rospy.Duration(movetime))]  

            self.client.send_goal(g)
            self.client.wait_for_result()
        except KeyboardInterrupt:
            self.client.cancel_goal()
            raise
        except:
            raise  
        
    def move_xc(self, control, movetime):
        controls = np.array(control).astype(int)
        assert controls.ndim == 1

        
        pos_list = []
        currpos = self.xc.get_currpos()
        goal_pos  = control * np.array([1,-1,-1,1]) + self.initpos

        resolution = 5
        splitnum = np.linalg.norm(goal_pos - currpos)/resolution

        interpolate_pos = np.linspace(currpos,goal_pos,int(splitnum))
        currpos = goal_pos
        pos_list.append(interpolate_pos)
        
        pos_array = np.vstack(pos_list)

        if len(pos_array) > 0:
            unit_sleep = movetime/len(pos_array)
            for pos in pos_array:
                self.xc.set_goalpos(pos)
                time.sleep(unit_sleep)


    def multiple_move_xc(self, controls, movetimes):
        controls = np.array(controls).astype(int)
        movetimes = np.array(movetimes)
        assert controls.ndim == 2
        assert movetimes.ndim == 1
        
        pos_list = []
        currpos = self.xc.get_currpos()

        for control in controls:
            goal_pos  = control * np.array([1,-1,-1,1]) + self.initpos

            resolution = 5
            splitnum = np.linalg.norm(goal_pos - currpos)/resolution

            interpolate_pos = np.linspace(currpos,goal_pos,int(splitnum))
            currpos = goal_pos
            pos_list.append(interpolate_pos)
            
        # pos_array = np.vstack(pos_list)
        for pos, movetime in zip(pos_list, movetimes):
            unit_sleep = movetime/len(pos)
            for pos_ in pos:
                self.xc.set_goalpos(pos_)
                time.sleep(unit_sleep)

    def execute_arm_speed(self, joints, speed):
        try:
            self.client = actionlib.SimpleActionClient('follow_joint_trajectory', FollowJointTrajectoryAction)
            print("Waiting for server...")
            self.client.wait_for_server()
            print("Connected to server")
            """ Initialize """
            self.move_arm_speed(joints, speed)
            print("Finish plan")

        except KeyboardInterrupt:
            rospy.signal_shutdown("KeyboardInterrupt")
            raise      


['/home/terry/Rilab/sOftrobot/SORO-Dynamixel-python', '', '/home/terry/catkin_ws/devel/lib/python3/dist-packages', '/opt/ros/noetic/lib/python3/dist-packages', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '/home/terry/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '.']


## INSTANTIATE Robot 

In [5]:
robot = OneFinger()

[SOFTROBOT] INSTANTIATED AT [b'/dev/ttyUSB0']


In [6]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90, 0]
robot.execute_arm_speed(init_joint, speed=1e-1)

Waiting for server...
Connected to server
Finish plan


In [7]:
currpos = robot.init_xc([4722, 3082,  957, 1628])

[SOFTROBOT] CONNECT
 SUCCEEDED TO OPEN THE PORT!
 SUCCEEDED TO CHANGE THE BAUDRATE!
COMMUNICATION SUCCESS
[SOFTROBOT] DETECT ID
[SOFTROBOT] [4]MOTORS DETECTED 
[SOFTROBOT] ID_LIST:[[13, 14, 15, 16]]
TORQUE ON
INITIALIZE POSITION
[4675 3105  985 1591]


In [8]:
# robot.move_xc([1000,0,0,0], movetime=5)

In [10]:
def move_soro(robot:OneFinger, motor_array, time_array):
    motor_array = np.array(motor_array)

    assert motor_array.ndim == 2
    assert len(time_array) == len(motor_array)

    motor_list = []
    currpos = robot.xc.get_currpos()
    
    for motor_control in motor_array:
        goal_pos  = PROPORTION_MATRIX @ motor_control * np.array([1,-1,-1,1]) + robot.initpos

        resolution = 10
        splitnum = np.linalg.norm(goal_pos - currpos)/resolution

        interpolate_pos = np.linspace(currpos,goal_pos,int(splitnum))
        currpos = goal_pos
        motor_list.append(interpolate_pos)
    
    from tqdm.notebook import tqdm
    i = 0
    for motor_array, movetime in tqdm(zip(motor_list, time_array)):
        
        unit_sleep = movetime/len(motor_array)
        for motor in motor_array:
            robot.xc.set_goalpos(motor)
            time.sleep(unit_sleep* 0.7)
        robot.client.wait_for_result()





# get qs and motor for paint

In [10]:
# q0 = [0,-DEG90,DEG90,-DEG90,-DEG90, 0]
# motor0 = [0,0,0,0]

# q1 = np.array([-0.6, -57.6, 109.6, -179.4, -100.5, 0.0])/180 * PI
# motor1 = [0,1000,0,0]
# q2 = np.array([-3.6, -56.0, 108.3, -177.1, -97.4, 0.0])/180 * PI
# moto2 = [1000,0,0,0]

# qs_array = np.array([q0,q1,q2])
# motor_array = np.array([motor0,motor1,moto2])

# qs_array.shape, motor_array.shape

In [11]:
q_ink_up   = np.array([-52.8, -79.8-1, 110.9, -121.3, -89.9, -52.7]) / 180 * PI
q_ink_down = np.array([-52.8, -74.8-1, 115.0, -130.3, -89.9, -52.7]) / 180 * PI

m_ink_up   = np.array([0,0,500,0])
m_ink_down = np.array([0,0,0,500])

q_ink_array = np.vstack([q_ink_up, q_ink_down, q_ink_down, q_ink_up])
m_ink_array = np.vstack([m_ink_up, m_ink_up, m_ink_down, m_ink_down])


In [12]:
# move_arm_soro(robot, q_ink_array, m_ink_array, time_array=2*np.ones(len(m_ink_array)).astype(np.float32))

In [19]:
pertub = [2,3,0,1]

motor_array = np.load("planned_traj/save/paint/motor_array.npy").astype(np.float32)[:, pertub]
qs_array    = np.load("planned_traj/save/paint/qs_array.npy").astype(np.float32)

assert (qs_array <  PI).all() and (qs_array>-PI).all()

In [20]:
# i = 1
# move_arm_soro(robot,np.array([qs_array[i]]), np.array([motor_array[i]]), time_array=np.ones(1).astype(np.float32))
# motor_array[i]

In [21]:
# motor_array = 200* np.ones_like(motor_array).astype(np.float32)

In [22]:
motor_array

array([[ 618.4649 ,  691.57635,  937.16876,  279.90594],
       [ 543.40295,  834.30255,  936.04034,  318.74585],
       [ 493.20248, 1034.622  ,  899.9741 ,  373.40823],
       [ 493.20248, 1034.622  ,  899.9741 ,  373.40823],
       [ 489.09375, 1050.942  ,  886.35565,  387.9576 ],
       [ 489.0934 , 1050.9421 ,  886.3557 ,  387.95737],
       [ 489.0934 , 1050.9421 ,  886.3557 ,  387.95737],
       [ 555.53033,  803.857  ,  946.15155,  326.1091 ],
       [ 652.9683 ,  629.6859 ,  939.26404,  277.35257],
       [ 652.9683 ,  629.6859 ,  939.26404,  277.35257],
       [ 653.2155 ,  629.25037,  939.35645,  277.13367],
       [ 653.21625,  629.24884,  939.3566 ,  277.13303],
       [ 653.21625,  629.24884,  939.3566 ,  277.13303],
       [ 603.7238 ,  730.78656,  882.1786 ,  355.89212],
       [ 608.1578 ,  794.5985 ,  762.6408 ,  456.77808],
       [ 608.1578 ,  794.5985 ,  762.6408 ,  456.77808],
       [ 606.87946,  802.48737,  759.7019 ,  461.28088],
       [ 606.4891 ,  804.8964 ,

In [24]:
len(motor_array)//3

63

In [25]:
(189) / 3, 64 * 3

(63.0, 192)

In [26]:
q_down = 2

assert len(motor_array) % 3 == 0


move_arm_soro(robot, q_ink_array, m_ink_array, time_array=0.5*np.ones(len(m_ink_array)).astype(np.float32))

for i in range(len(motor_array)//3):
    if i>38: break
    
    qs_array_ = qs_array[i*3:(i+1)*3] + np.array([0, q_down,-q_down ,0,0,0]) / 180 * PI
    motor_array_ = motor_array[i*3:(i+1)*3]
    # qs_array_ = qs_array[n:m] + np.array([0, q_down,-q_down ,0,0,0]) / 180 * PI
    # motor_array_ = motor_array[n:m]


    if i% (4*2)==7:
        move_arm_soro(robot, q_ink_array, m_ink_array, time_array=0.5*np.ones(len(m_ink_array)).astype(np.float32))
        
    move_arm_soro(robot, qs_array_, motor_array_, time_array=0.5 * np.ones(len(motor_array_)).astype(np.float32))

move_arm_soro(robot, q_ink_array, m_ink_array, time_array=0.5*np.ones(len(m_ink_array)).astype(np.float32))


0it [00:00, ?it/s]

[-0.92153385 -1.41022604  1.93557014 -2.11708438 -1.569051   -0.91978852]
[-0.92153385 -1.32295957  2.00712864 -2.27416402 -1.569051   -0.91978852]


/tmp/ipykernel_128504/1366246583.py:57: RuntimeWarning: divide by zero encountered in double_scalars
  unit_sleep = movetime/len(motor_array)


[-0.92153385 -1.32295957  2.00712864 -2.27416402 -1.569051   -0.91978852]
[-0.92153385 -1.41022604  1.93557014 -2.11708438 -1.569051   -0.91978852]


0it [00:00, ?it/s]

...

[-0.30640203 -1.01396159  1.56113641 -2.11797023 -1.57079637 -0.30640155]
[-0.33860871 -1.00945298  1.5555257  -2.11686993 -1.57079685 -0.3386091 ]
[-0.36909327 -1.00297741  1.54500548 -2.11282516 -1.57079697 -0.36909372]


0it [00:00, ?it/s]

[-0.36909327 -1.00297741  1.54500548 -2.11282516 -1.57079697 -0.36909372]
[-0.36143255 -1.03162448  1.51767639 -2.05685019 -1.57079649 -0.36143318]
[-0.35361522 -1.06121425  1.49113826 -2.00072145 -1.57079637 -0.3536157 ]


0it [00:00, ?it/s]

[-0.92153385 -1.41022604  1.93557014 -2.11708438 -1.569051   -0.91978852]
[-0.92153385 -1.32295957  2.00712864 -2.27416402 -1.569051   -0.91978852]
[-0.92153385 -1.32295957  2.00712864 -2.27416402 -1.569051   -0.91978852]
[-0.92153385 -1.41022604  1.93557014 -2.11708438 -1.569051   -0.91978852]


In [27]:
pertub = [2,3,0,1]

motor_array = np.load("planned_traj/paint/motor_array.npy").astype(np.float32)[:, pertub]
qs_array    = np.load("planned_traj/paint/qs_array.npy").astype(np.float32)

assert (qs_array <  PI).all() and (qs_array>-PI).all()

In [28]:
q_down = 2

assert len(motor_array) % 3 == 0


for i in range(len(motor_array)//3):
    qs_array_ = qs_array[i*3:(i+1)*3] + np.array([0, q_down,-q_down ,0,0,0]) / 180 * PI
    motor_array_ = motor_array[i*3:(i+1)*3]
    # qs_array_ = qs_array[n:m] + np.array([0, q_down,-q_down ,0,0,0]) / 180 * PI
    # motor_array_ = motor_array[n:m]


    if i% (4*4)==15:
        move_arm_soro(robot, q_ink_array, m_ink_array, time_array=0.5*np.ones(len(m_ink_array)).astype(np.float32))
        
    move_arm_soro(robot, qs_array_, motor_array_, time_array=0.5 * np.ones(len(motor_array_)).astype(np.float32))

move_arm_soro(robot, q_ink_array, m_ink_array, time_array=0.5*np.ones(len(m_ink_array)).astype(np.float32))


0it [00:00, ?it/s]

[-0.36485094 -0.91027979  1.24818675 -1.90870571 -1.57079482 -0.36485311]
[-0.35797149 -0.88595543  1.27377025 -1.95861006 -1.57079589 -0.35797143]
[-0.35220963 -0.85849134  1.29775039 -2.01005435 -1.57079637 -0.35220963]


0it [00:00, ?it/s]

[-0.35220963 -0.85849134  1.29775039 -2.01005435 -1.57079637 -0.35220963]
[-0.3375732  -0.84678875  1.27986781 -2.00387549 -1.57079589 -0.33757401]
[-0.32251281 -0.83742891  1.26350061 -1.99686873 -1.57079566 -0.32251349]


0it [00:00, ?it/s]

...

[-0.37071729 -0.98996107  1.52211789 -2.10295177 -1.57079637 -0.37071723]
[-0.34980482 -0.98046557  1.5083584  -2.09868932 -1.57079589 -0.34980527]
[-0.32896769 -0.97161202  1.49373655 -2.09292102 -1.57079566 -0.3289682 ]


0it [00:00, ?it/s]

[-0.32896769 -0.97161202  1.49373655 -2.09292102 -1.57079566 -0.3289682 ]
[-0.33466998 -1.00500993  1.47570435 -2.04149199 -1.57079577 -0.33467007]
[-0.34404951 -1.0352482   1.45153574 -1.98708522 -1.57079577 -0.3440496 ]


0it [00:00, ?it/s]

[-0.92153385 -1.41022604  1.93557014 -2.11708438 -1.569051   -0.91978852]
[-0.92153385 -1.32295957  2.00712864 -2.27416402 -1.569051   -0.91978852]
[-0.92153385 -1.32295957  2.00712864 -2.27416402 -1.569051   -0.91978852]
[-0.92153385 -1.41022604  1.93557014 -2.11708438 -1.569051   -0.91978852]


In [ ]:
q0 = init_joint
q1 = np.array([0, -44, 63, -139, -92, 0]) / 180 * PI
q2 = np.array([0, -33, 58, -150, -92, 0])/180 * PI
q3 = np.array([0, -76, 124, -150, -93, 11])/180 * PI

motor_0 = [0,0,0,0]
motor_1 = [0,0,0,1000]
motor_2 = [0,0,1000,0]
motor_3 = [0,0,1000,0]


In [ ]:
qs_array = np.array([q0, q1, q2, q3, q0])
motor_array = np.array([motor_0, motor_1, motor_2, motor_3, motor_0])
time_array = np.array([1,1,3,1,1])
# move_arm_soro(robot, qs_array, motor_array, time_array=time_array)

qs_array.shape, motor_array.shape,


In [ ]:
motor_array[:,[2,3,0,1]]

# 1 -> 3
# 2 -> 4
# 3 -> 1
# 4 -> 2

In [ ]:
pertub = [2,3,0,1]

motor_array = np.load("planned_traj/scraps/motor_array.npy")[:,0,:].astype(np.float32)[:, pertub]
qs_array = np.load("planned_traj/scraps/qs_array.npy").astype(np.float32)

qs_array.shape, motor_array.shape

In [ ]:
move_arm_soro(robot, qs_array, motor_array, time_array=np.ones(len(motor_array)).astype(np.float32))

In [ ]:
        
robot.execute_arm_speed(init_joint, speed=1e-1)
robot.move_xc(np.array([0,0,0,0]), movetime=3)        

motor_array = np.load("planned_traj/scraps/motor_array.npy")[:,0,:] * 1.5
qs_array    = np.load("planned_traj/scraps/qs_array.npy") + np.array([0,0,0,-4*DEG90,0, DEG90])
# motor_array = np.array([[1000,0,0,0]]*30)
qs_array.shape, motor_array.shape

move_arm_soro(robot, qs_array, motor_array, time_array=np.ones(len(qs_array))*2)


In [ ]:
# import numpy as np

motor_controls = np.load("planned_traj/scraps/motor_array.npy")[:,0,:] * 1.5
motor_controls.shape
robot.multiple_move_xc(motor_controls, movetimes=np.ones(len(motor_controls))*0.5 )

In [ ]:
import numpy as np

qs_array = np.load("planned_traj/scraps/qs_array.npy")
qs_array[0] - init_joint


# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90, 0]
# robot.execute_arm(init_joint, movetime = 5)

# for qs in qs_array:
#     robot.execute_arm(qs + np.array([0,0,0,-4*DEG90,0,-DEG90]), movetime = 10)

# Calibrate initpos
반드시 Cap을 벗기고 진행할것

In [ ]:
# Upright_joint = np.array([0, -90+45, 90-45, -90, 90, 90])/90*DEG90
# robot.execute_arm(Upright_joint)

In [ ]:
# robot.init_xc(initpos=np.array([None]))
# initpos = robot.xc.get_currpos()

In [ ]:
# relaxed_pos = initpos

In [ ]:
# initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
#        1990,  2605, -703, 2454, 1732])
# # initpos[-4:] = relaxed_pos[-4:]
# robot.init_xc(initpos=initpos)

In [ ]:
# relaxed_pos

In [ ]:
# # 1번은 플러스, 2번 마이너스, 3번 마이너스, 4번 플러스
# pos = initpos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,0, 100,200,200,200]) * np.array([1,-1,-1,1]*4)
# # robot.move_xc(pos)

# pos = pos + PROPORTION_MATRIX @ np.array([0,0,0,0, 0,0,0,0, 0,0,0,0, 600,600,600,600]) * np.array([1,-1,-1,1]*4)

# robot.move_xc(pos)

# robot.xc.get_currpos()

In [ ]:
# initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
#        1990, 4110,  970, 3106, 3197])
# robot.init_xc(initpos)
# robot.xc.get_currpos()

# pos = initpos + np.array([300]*16) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)

# pos = initpos + PROPORTION_MATRIX @ np.array([1000,0,0,0, 0,0,1000,0, 0,1000,0,0, 0,0,0,1000]) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)


# pos = initpos + PROPORTION_MATRIX @ np.array([0,1000,0,0, 0,0,0,1000, 1000,0,0,0, 0,0,1000,0]) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)

# robot.move_xc(initpos)

# Twist Open

In [ ]:
robot.init_xc(np.array([None]))
initpos = robot.xc.get_currpos()
pos = initpos + np.array([1,1,1,1, 1,1,1,1, 1,1,1,1, 1,1,1,1])*1000 * np.array([1,-1,-1,1]*4)
robot.move_xc(pos)

initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
       1990,  2705, -803, 2354, 1832])
robot.init_xc(initpos)

robot.xc.get_currpos()

# Close Up

In [ ]:
Target_up_joint = [0.001,-DEG90,DEG90,-DEG90,-DEG90,DEG90]

Target_down_joint = np.array([0.02,-86.79,106.58,-109.79, -90.04, 90.11])/90*DEG90

Target_down_joint2 = np.array([0.02,-87.48,104.85,-107.46, -90.03, 90.09])/90*DEG90
Target_down_joint = (Target_down_joint2+Target_down_joint)/2
assert (Target_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint2 * np.array([1,-1,1,-1,-1,1])>0).all()

robot.execute_arm(Target_up_joint)
robot.pmove_xc(np.array([0,0,0,0]*4))



In [ ]:
coeff = 1.4
rollout_actuation = np.load("actuations/Custom_grasp.npy") * coeff

# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

In [ ]:
# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

robot.execute_arm(Target_down_joint)

# 접근 동작
act = rollout_actuation[1].tolist()
robot.pmove_xc(np.array(act*4))

# 잡는 동작
act = rollout_actuation[2].tolist()
robot.pmove_xc(np.array(act*4))

# 절반 돌리고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))

# 나머지 돌리고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))

# 들고
act = rollout_actuation[5].tolist()
robot.pmove_xc(np.array(act*4))

# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

# 접근 동작
act = rollout_actuation[1].tolist()
robot.pmove_xc(np.array(act*4))

# 잡는 동작
act = rollout_actuation[2].tolist()
robot.pmove_xc(np.array(act*4))

# 절반 돌리고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))

# 나머지 돌리고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))

# 들고
act = rollout_actuation[5].tolist()
robot.pmove_xc(np.array(act*4))

# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

# 접근 동작
act = rollout_actuation[1].tolist()
robot.pmove_xc(np.array(act*4))

# 잡는 동작
act = rollout_actuation[2].tolist()
robot.pmove_xc(np.array(act*4))

# 절반 돌리고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))

# 나머지 돌리고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))



In [ ]:
robot.execute_arm(Target_up_joint)
time.sleep(4)
robot.execute_arm(Target_down_joint2)


In [ ]:
### Capping ###

# 놓고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))


# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))


# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))


In [ ]:
robot.execute_arm(Target_up_joint)

In [ ]:
# # Ungrasp
# act = rollout_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)

# # 접근 동작
# act = rollout_actuation[1].tolist()
# robot.pmove_xc(np.array(act*4))

# # 잡는 동작
# act = rollout_actuation[2].tolist()
# robot.pmove_xc(np.array(act*4))

# # 절반 돌리고
# act = rollout_actuation[3].tolist()
# robot.pmove_xc(np.array(act*4))

# # 나머지 돌리고
# act = rollout_actuation[4].tolist()
# robot.pmove_xc(np.array(act*4))

In [ ]:
robot.pmove_xc(np.array([1000,0,1000,0]*4))

In [ ]:
# # 잡는 동작
# robot.pmove_xc(np.array([0,1800,200,800]*4))

# # 절반 돌리고
# robot.pmove_xc(np.array([800,800,500,1500]*4))

# # 나머지 돌리고
# robot.pmove_xc(np.array([500,800,0,1800]*4))

In [ ]:
coeff = 0.7
rollout_actuation = np.load("actuations/Custom_grasp.npy") * coeff

cap_actuation = np.zeros_like(rollout_actuation)

cap_actuation[:,0] = rollout_actuation[:,2]
cap_actuation[:,1] = rollout_actuation[:,3]
cap_actuation[:,2] = rollout_actuation[:,0]
cap_actuation[:,3] = rollout_actuation[:,1]


In [ ]:
# # robot.execute_arm(Target_down_joint)

# # Ungrasp
# act = cap_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)

# # robot.execute_arm(Target_down2_joint)

# # 접근 동작
# act = cap_actuation[1].tolist()
# robot.pmove_xc(np.array(act*4))

# # 잡는 동작
# act = cap_actuation[2].tolist()
# robot.pmove_xc(np.array(act*4))

# # 절반 돌리고
# act = cap_actuation[3].tolist()
# robot.pmove_xc(np.array(act*4))

# # 나머지 돌리고
# act = cap_actuation[4].tolist()
# robot.pmove_xc(np.array(act*4))

# # 들고
# act = cap_actuation[5].tolist()
# robot.pmove_xc(np.array(act*4))

# # Ungrasp
# act = cap_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)


In [ ]:
# 놓고
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

In [ ]:
coeff = 1.5
rollout_actuation = np.load("actuations/Custom_grasp.npy") * coeff

# 놓고
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

# robot.execute_arm(Target_down_joint)

# 들고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))

# # 원래대로 오고
# act = rollout_actuation[4].tolist()
# robot.pmove_xc(np.array(act*4))

# # 절반 돌리고
# act = rollout_actuation[3].tolist()
# robot.pmove_xc(np.array(act*4))

# # 나머지 돌리고
# act = rollout_actuation[2].tolist()
# robot.pmove_xc(np.array(act*4))

# # 들고
# act = rollout_actuation[1].tolist()
# robot.pmove_xc(np.array(act*4))

# # 놓고
# act = rollout_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)


In [ ]:
rollout_actuation = np.load("actuations/Custom_grasp.npy")
rollout_actuation = np.flip(rollout_actuation,axis=0)
rollout_actuation

In [ ]:

act = rollout_actuation[5].tolist()
robot.pmove_xc(np.array(act*4))

In [ ]:
# robot.pmove_xc(np.array([0,0,0,0]*4))

# Jar

In [ ]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [ ]:
# robot.execute_arm(Place_up_joint)
# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
# robot.execute_arm(init_joint)

In [ ]:
Custom_grasp = np.load("actuations/Custom_grasp.npy")
Custom_grasp[0] = Custom_grasp[0]* 1.2
Custom_grasp[1:] = Custom_grasp[1:]* 1.6


Custom_grasp = Custom_grasp.tolist()
Custom_grasp

In [ ]:
robot.pmove_xc(np.tile(Custom_grasp[0],4))

In [ ]:
for actuation in Custom_grasp:
    time.sleep(1)
    robot.pmove_xc(np.array(np.tile(actuation,4)))

for actuation in Custom_grasp:
    time.sleep(1)
    if actuation == Custom_grasp[-1]: break
    robot.pmove_xc(np.array(np.tile(actuation,4)))
    

In [ ]:
actuation == Custom_grasp[-1]

In [ ]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-51.04,-79.42,73.24,-83.68,-89.61,81.53])/90*DEG90
Target_down_joint = np.array([-50.75,-75,100.9,-115.94,-89.4,82.05])/90*DEG90
Place_up_joint = np.array([-38.01,-54.74,37.94,-73.06,-89.60,94.45])/90*DEG90
Place_down_joint = np.array([-38.07,-49.32,77.91,-118.45,-89.65,94.72])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


# robot.execute_arm(init_joint)

# robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

# robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# # 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# robot.execute_arm(Target_up_joint)
# robot.execute_arm(Place_up_joint)
# robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
# robot.execute_arm(Place_up_joint)
# robot.execute_arm(init_joint)

# Tumbler

In [ ]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [ ]:
# robot.execute_arm(Place_up_joint)
# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
# robot.execute_arm(init_joint)

In [ ]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-34.38,-90.85,93.59,-92.60,-89.61,98.27])/90*DEG90
Target_down_joint = np.array([-34.42,-87.67,107.73,-109.93,-89.65,98.36])/90*DEG90
Place_up_joint = np.array([-25.41,-66.39,63.93,-87.40,-89.59,107.17])/90*DEG90
Place_down_joint = np.array([-25.45,-56.45,89.79,-123.20,-89.68,107.37])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# # 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


# Tumbler2

In [ ]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [ ]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-11.76,-94.52,93.83,-89.17,-89.60,120.91])/90*DEG90
Target_down_joint = np.array([-11.76,-91.46,111.03,-109.48,-89.64,121.01])/90*DEG90
Place_up_joint = np.array([-8.21,-66.55,61.03,-84.34,-89.52,124.35])/90*DEG90
Place_down_joint = np.array([-8.26,-57.02,90.09,-122.93,-89.67,124.56])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))


robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


## Gat

In [ ]:
robot.pmove_xc(np.array([0,0,0,0]*4))


In [ ]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint   = np.array([16.84,-83.79,82.93,-89.01,-89.59,149.49])/90*DEG90
Target_down_joint = np.array([16.81,-82.13,96.38,-104.12,-89.61,149.57])/90*DEG90
Place_up_joint    = np.array([12.25,-55.41,42.79,-77.25,-89.58,144.78])/90*DEG90
Place_down_joint  = np.array([12.20,-48.98,77.21,-118.10,-89.63,145.01])/90*DEG90

assert (Target_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)


# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1500,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,1200,1600]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))


robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


In [ ]:
robot.pmove_xc(np.array([0,0,0,0]*4))

# MAYO

In [ ]:
pos = initpos+ PROPORTION_MATRIX @ np.array([1,0,1,0, 0,0,0,0, 0,0,0,0, 1,0,1,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0.3,0,1,0, 1,0,0,0.3, 0,0,0,0]) * 4000 * np.array([1,-1,-1,1]*4)

# pos = pos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,400, 0,0,0,0]) * np.array([1,-1,-1,1]*4)
snapbot.set_goalposcluster(pos,1)

In [ ]:
pos = initpos+ PROPORTION_MATRIX @ np.array([1,0,1,0, 0,0,0,0, 0,0,0,0, 1,0,1,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0.3,0,1,0, 1,0,0,0.3, 0,0,0,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0,0,1,0, 1,0,0,0, 0,0,0,0]) * 1000 * np.array([1,-1,-1,1]*4)

# pos = pos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,400, 0,0,0,0]) * np.array([1,-1,-1,1]*4)
snapbot.set_goalposcluster(pos,1)